In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

In [9]:
#read in df with dummies
final_df = pd.read_csv('../data/final_df.csv')

C:\Users\Conrad\AppData\Local\Temp\ipykernel_24404\1325653610.py:1: DtypeWarning: Columns (5,6,9,10,12,13,14,15,16,17,18,19,21,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  final_df = pd.read_csv('../data/final_df.csv')


In [11]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [12]:
final_df

,EventTimeStamp,ESS_Id,EquipmentID,spn,fmi,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,Aftertreatment System,Ambient Air Temperature,Anti-Lock Braking (ABS) Controller,Auxiliary Input/Output,Auxiliary Temperature Sensor Input Circuit,Barometric Pressure Sensor,Battery,Crankcase,DAS Module to ECM Communication,ECU Power Output Supply Voltage,Electric Lift Pump,Electrical Charging System Voltage,Electronic Fuel Injection Control Valve,Electronic Throttle Control,Engine Air Filter,Engine Brake,Engine Charge Air Cooler Outlet Temperature,Engine Control Module,Engine Coolant,Engine Cooling System,Engine Diesel Particulate Filter,Engine Exhaust Back Pressure,Engine Fuel,Engine Injector Metering Rail,Engine Intake Air Heater,Engine Misfire,Engine Oil,Engine Position Sensor,Engine Protection Torque Full Derate,Engine Protection Torque Partial Derate,Engine Speed,Engine Starter Motor Relay,Engine Total Idle Hours,Exhaust Gas,Exhaust Gas Recirculation System,Fan Control,Fuel Control Module,High Pressure Common Rail Fuel Pressure Relief Valve,Indicator Lamp,Injector Metering Rail,Injector Solenoid,Intake Manifold,Intake Throttle,J1939 Network,Real Time Clock,Sensor Supply,Steering Wheel Angle Sensor,Total Vehicle Distance,Transmission Communications Failure Idle Torque Limit,Transmission Oil,Turbocharger,Unknown,Variable Geometry Turbocharger,Vehicle Identification Number,Wheel-Based Vehicle Speed
0,2015-05-11 13:11:20,2363162,301,639,2,14.4,13.92,False,0.00000,120841.7,185.0,18.0,55.1,193.6062,1145.75,2372.75,76.4,17070.79800024,2.760607,134.6,True,154.4,1279,False,4.475814,3276.75,14.4,0.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-05-13 08:22:32,2400445,301,596,31,89.6,14.4275,False,0.00000,121687.1,186.8,38.0,52.78,217.175,1257.5,2388.15,72.4,17193.373832368,6.326941,100.4,True,120.2,4351,False,64.63717,3276.75,89.6,4.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-05-18 09:34:05,2482983,301,3226,10,0.0,14.355,False,0.00000,123057.1,185,0.0,55.68,213.9688,1230.25,2413.65,50.8,17376.312978378,0,120.2,True,118.4,17407,False,63.52549,3276.75,0.0,20.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2015-05-21 13:57:35,2554027,301,639,2,12.0,14.5725,False,0.00000,124496.5,183.2,14.0,51.62,196.7,1115.5,2443.5,41.2,17577.34790995,2.311513,109.4,True,107.6,1279,False,4.364162,3276.75,12.0,1.45,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2015-05-28 13:31:41,2661725,301,596,31,67.2,13.92,False,0.00000,125989.2,183.2,68.0,51.62,215.2063,1257.5,2475.9,62.4,17776.401551132,11.4651,123.8,True,129.2,4351,False,64.8362,3276.75,68.0,20.01,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [16]:
#Get list of unique EquipmentIDs
eq_ids = final_df.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [19]:
#create a column of random numbers from 0-1
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

eq_ids

,EquipmentID,random
0,301,0.6682
431,302,0.4220
2049,303,0.5929
2786,304,0.9617
4023,305,0.8360
...,...,...
554833,2375,0.7059
554838,2376,0.1220
554845,2377,0.4415
554853,2380,0.5786


In [20]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [23]:
#merge modeling column onto original dataframe
final_df = final_df.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

final_df = final_df.merge(eq_ids, on='EquipmentID')

In [30]:
#created training dataframe
df_training = final_df[final_df['modeling']=='training']
df_training = df_training.drop(columns=['modeling', 'random'])
df_training

,EventTimeStamp,ESS_Id,EquipmentID,spn,fmi,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,Aftertreatment System,Ambient Air Temperature,Anti-Lock Braking (ABS) Controller,Auxiliary Input/Output,Auxiliary Temperature Sensor Input Circuit,Barometric Pressure Sensor,Battery,Crankcase,DAS Module to ECM Communication,ECU Power Output Supply Voltage,Electric Lift Pump,Electrical Charging System Voltage,Electronic Fuel Injection Control Valve,Electronic Throttle Control,Engine Air Filter,Engine Brake,Engine Charge Air Cooler Outlet Temperature,Engine Control Module,Engine Coolant,Engine Cooling System,Engine Diesel Particulate Filter,Engine Exhaust Back Pressure,Engine Fuel,Engine Injector Metering Rail,Engine Intake Air Heater,Engine Misfire,Engine Oil,Engine Position Sensor,Engine Protection Torque Full Derate,Engine Protection Torque Partial Derate,Engine Speed,Engine Starter Motor Relay,Engine Total Idle Hours,Exhaust Gas,Exhaust Gas Recirculation System,Fan Control,Fuel Control Module,High Pressure Common Rail Fuel Pressure Relief Valve,Indicator Lamp,Injector Metering Rail,Injector Solenoid,Intake Manifold,Intake Throttle,J1939 Network,Real Time Clock,Sensor Supply,Steering Wheel Angle Sensor,Total Vehicle Distance,Transmission Communications Failure Idle Torque Limit,Transmission Oil,Turbocharger,Unknown,Variable Geometry Turbocharger,Vehicle Identification Number,Wheel-Based Vehicle Speed
6566,2018-03-01 08:23:16,41365867,306,639,2,25.6,14.2825,False,45.36010,280706.2,179.6,15.0,37.12,202.0437,971.25,8207.05,74.8,38713.75379047,1.294447,32.0,True,109.4,1279,False,5.038932,NaN,0.0,2.03,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6567,2018-03-02 11:31:02,41497176,306,639,2,30.0,14.5725,False,39.76776,281138.6,183.2,10.0,40.02,199.7937,1258.875,8221.3,82.0,38778.872201288,1.334073,32.0,True,96.8,1279,False,9.058427,NaN,0.0,1.16,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6568,2018-03-07 09:06:30,41902367,306,96,3,0.0,14.4275,False,55.30204,281730.5,181.4,8.0,23.2,194.9563,602.125,8249.5,58.4,38868.558612942,0.4226766,32.0,True,113.0,1279,True,0.0,NaN,0.0,0.29,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6569,2018-03-07 09:06:30,41902366,306,829,3,0.0,14.4275,False,55.30204,281730.5,181.4,8.0,23.2,194.9563,602.125,8249.5,58.4,38868.558612942,0.4226766,32.0,True,113.0,1279,True,0.0,NaN,0.0,0.29,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6570,2018-03-07 10:26:41,41915375,306,96,3,0.0,14.355,False,55.30204,281757.1,188.6,10.0,19.14,213.7437,601.125,8250.8,63.6,38873.445795904,0.409468,32.0,True,96.8,1279,True,0.0,NaN,0.0,0.29,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [32]:
#create dataframe of both training and testing to have large test dataset for testing. 
#With size of dataset, probably can just keep it confined to modeling == 'testing'
df_train_test = final_df[final_df['modeling']!='validation']
df_train_test = df_train_test.drop(columns=['modeling', 'random'])
df_train_test.shape

(403609, 83)

In [33]:
#final dataframe without modeling or random column for final predictions
df_total = final_df.drop(columns=['modeling', 'random'])

In [54]:
#setting up for model testing with training dataset. Realized here about how may need to adjust target variable 
#to implement oversampling
features = df_total.iloc[:,5:].columns.tolist()

target =['spn']

X_train = df_training[features]
y_train = df_training[target]

In [60]:
logreg = LogisticRegression()

In [61]:
#u
logreg.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').